# Training a CRF with DataTurks Annotated .tsv File.

In [12]:
%reset -f

In [13]:
import pandas as pd
import re
import numpy as np
import random
import time
from seqeval.metrics import f1_score,classification_report
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional,Input,concatenate,SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from nltk import pos_tag
import eli5
from sklearn_crfsuite import CRF
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite.metrics import flat_f1_score

## Formatting Data to Sequences

### Contents:

1. Reading File and formatting as sequences
2. Formatting Entities to IOB Scheme
3. Adding POS and Other CRF Features
4. Train Test Split
5. Training Model
6. Evaluating Model Preformance
7. Saving Model Results

### 1. Reading and Formatting File:

In [14]:
def read_turks(file):
    with open(file) as f:
        lines = [i.rstrip().split("\t") for i in f.readlines()]
    return lines

In [44]:
file = "./data/Medical NER V2 2000.tsv"
word_ents = read_turks(file)
word_ents[:20]

[['"In', 'O'],
 ['the', 'O'],
 ['Hospital3', 'GEO'],
 ['Emergency', 'GEO'],
 ['Room', 'GEO'],
 [',', 'O'],
 ['her', 'O'],
 ['oxygen', 'O'],
 ['saturation', 'O'],
 ['was', 'O'],
 ['100%', 'O2 Saturation'],
 ['on', 'O2 Saturation'],
 ['CPAP', 'O2 Saturation'],
 ['"', 'O'],
 [''],
 ['She', 'O'],
 ['was', 'O'],
 ['not', 'DOS'],
 ['able', 'DOS'],
 ['to', 'DOS']]

In [45]:
def clean_words(word_ents):
    '''removes quote and comma characters from'''
    new_word_ents = []
    for ents in word_ents:
        word = ents[0]
        word = word.replace('"','')
        ents[0] = word
        new_word_ents.append(ents)
    return new_word_ents

In [46]:
new_ents = clean_words(word_ents)
new_ents[:20]

[['In', 'O'],
 ['the', 'O'],
 ['Hospital3', 'GEO'],
 ['Emergency', 'GEO'],
 ['Room', 'GEO'],
 [',', 'O'],
 ['her', 'O'],
 ['oxygen', 'O'],
 ['saturation', 'O'],
 ['was', 'O'],
 ['100%', 'O2 Saturation'],
 ['on', 'O2 Saturation'],
 ['CPAP', 'O2 Saturation'],
 ['', 'O'],
 [''],
 ['She', 'O'],
 ['was', 'O'],
 ['not', 'DOS'],
 ['able', 'DOS'],
 ['to', 'DOS']]

In [47]:
def create_seqs(word_ents):
    seqs = []
    seq = []
    for ents in word_ents:
        if len(ents)>1:
            if len(ents[0])>0:
                if ents[0][-1] == ".":
                    seq.append([ents[0][:-1],ents[1]])
                else:
                    seq.append(ents)
        else:
            seqs.append(seq)
            seq=[]
    return seqs

In [48]:
seqs = create_seqs(new_ents)
seqs[0]

[['In', 'O'],
 ['the', 'O'],
 ['Hospital3', 'GEO'],
 ['Emergency', 'GEO'],
 ['Room', 'GEO'],
 [',', 'O'],
 ['her', 'O'],
 ['oxygen', 'O'],
 ['saturation', 'O'],
 ['was', 'O'],
 ['100%', 'O2 Saturation'],
 ['on', 'O2 Saturation'],
 ['CPAP', 'O2 Saturation']]

### Expand Special Characters

In [49]:
def expand_word(ent):
    '''Splits at specified special characters keeping special characters as their own value'''
    words = re.split('([/\-\%><])',ent[0])
    return [[i,ent[1]] for i in words if len(i)>0]

def expand_special_chars(seqs):
    '''Expands special characters in words into seperate words while '''
    new_seqs = []
    for seq in seqs:
        new_seq = []
        for word in seq:
            new_seq += expand_word(word)
        new_seqs.append(new_seq)
    return new_seqs

In [50]:
ex_seqs = expand_special_chars(seqs)
ex_seqs[0]

[['In', 'O'],
 ['the', 'O'],
 ['Hospital3', 'GEO'],
 ['Emergency', 'GEO'],
 ['Room', 'GEO'],
 [',', 'O'],
 ['her', 'O'],
 ['oxygen', 'O'],
 ['saturation', 'O'],
 ['was', 'O'],
 ['100', 'O2 Saturation'],
 ['%', 'O2 Saturation'],
 ['on', 'O2 Saturation'],
 ['CPAP', 'O2 Saturation']]

### Encode Common Numeric Values

In [51]:
def encode_numerics(seq):
    '''Add encodings for common number types'''
    enc_seq = []
    for ent in seq:
        enc = ent[0].strip()
        if re.match("^\d$",ent[0]) != None:
            enc = "<1DigitNum>"
        elif re.match("^\d\d$",ent[0]) != None:
            enc = "<2DigitNum>"
        elif re.match("^\d\d\d$",ent[0]) !=None:
            enc = "<3DigitNum>"
        elif re.match("^\d{4}$",ent[0]) != None:
            enc = "4DigitNum"
        elif re.match("^\d*\.\d*$",ent[0]) != None:
            enc = "<DecimalNum>"
        elif re.match("^\d+,\d+$",ent[0]) != None:
            enc = "<CommaNum>"
        elif re.match("^\d+'?s$",ent[0]) !=None:
            enc = "<RangeNum>"
            
        enc_seq.append([enc,ent[1]])
    return enc_seq

In [52]:
enc_seqs = [encode_numerics(seq) for seq in ex_seqs]
enc_seqs[0]

[['In', 'O'],
 ['the', 'O'],
 ['Hospital3', 'GEO'],
 ['Emergency', 'GEO'],
 ['Room', 'GEO'],
 [',', 'O'],
 ['her', 'O'],
 ['oxygen', 'O'],
 ['saturation', 'O'],
 ['was', 'O'],
 ['<3DigitNum>', 'O2 Saturation'],
 ['%', 'O2 Saturation'],
 ['on', 'O2 Saturation'],
 ['CPAP', 'O2 Saturation']]

### 2. Formatting Entities to IOB (Inside,Outside, Beginning) Scheme 

In [53]:
def clean_tags(word_ents):
    '''adds IOB scheme to tags'''
    new_ents = []
    for i in range(0,len(word_ents)):
        if word_ents[i][1] == "O":
            tag = word_ents[i][1]
        else:
            if not i:
                tag = "B-"+word_ents[i][1]
            else:
                if (word_ents[i][1] != word_ents[i-1][1]):
                    tag = "B-"+word_ents[i][1]
                else:
                    tag = "I-"+word_ents[i][1]

        new_ents.append([word_ents[i][0],tag])
    return new_ents

In [54]:
cleaned_tag_seqs = [clean_tags(ents) for ents in enc_seqs]
cleaned_tag_seqs[3]

[['She', 'O'],
 ['subsequently', 'O'],
 ['underwent', 'O'],
 ['placement', 'B-Procedure'],
 ['of', 'I-Procedure'],
 ['two', 'I-Procedure'],
 ['silicone', 'I-Procedure'],
 ['stents', 'I-Procedure'],
 [',', 'O'],
 ['one', 'B-Quantity'],
 ['in', 'O'],
 ['the', 'B-BODY'],
 ['left', 'I-BODY'],
 ['main', 'I-BODY'],
 ['stem', 'I-BODY'],
 ['and', 'O'],
 ['one', 'B-Quantity'],
 ['in', 'O'],
 ['the', 'B-BODY'],
 ['trachea', 'I-BODY']]

### 3. Adding POS and Other CRF Specific Features

In [55]:
def add_pos(seqs):
    new_seqs = []
    for sentance in seqs:
        words = [word[0] for word in sentance]
        pos = pos_tag(words)        
        new_seq = [pos[i]+(sentance[i][1],) for i in range(len(sentance))]
        new_seqs.append(new_seq)
    return new_seqs

In [56]:
pos_seqs = add_pos(cleaned_tag_seqs)
print(pos_seqs[0])
print(len(pos_seqs))

[('In', 'IN', 'O'), ('the', 'DT', 'O'), ('Hospital3', 'NNP', 'B-GEO'), ('Emergency', 'NNP', 'I-GEO'), ('Room', 'NNP', 'I-GEO'), (',', ',', 'O'), ('her', 'PRP$', 'O'), ('oxygen', 'NN', 'O'), ('saturation', 'NN', 'O'), ('was', 'VBD', 'O'), ('<3DigitNum>', 'CD', 'B-O2 Saturation'), ('%', 'NN', 'I-O2 Saturation'), ('on', 'IN', 'I-O2 Saturation'), ('CPAP', 'NNP', 'I-O2 Saturation')]
2009


In [57]:
def word2features(sent, i):
    '''
    
    From:
    https://www.depends-on-the-definition.com/named-entity-recognition-conditional-random-fields-python/
    
    '''
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [58]:
x = [sent2features(s) for s in pos_seqs]
y = [sent2labels(s) for s in pos_seqs]

### 4. Train Test Split

In [59]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

### 5. Training Model

In [89]:
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [90]:
crf.fit(x_train, y_train)
pred = crf.predict(x_test)
tr_pred = crf.predict(x_train)

### 6. Evaluating Model Performance

In [91]:
report = classification_report(y_test,pred)
print("Test Results:\n","-"*60)
print(report)
print("-"*60)

Test Results:
 ------------------------------------------------------------
                            precision    recall  f1-score   support

                 Procedure       0.61      0.43      0.51        44
                       DOS       0.52      0.50      0.51       161
                      Drug       0.87      0.78      0.82        50
                       POI       1.00      0.60      0.75         5
                Heart Rate       0.90      0.82      0.86        11
        Patient Relocation       0.78      0.86      0.82        29
             O2 Saturation       0.86      0.71      0.77        17
          Respiratory Rate       0.89      0.80      0.84        10
          Accident / Event       0.40      0.15      0.22        13
                       GEO       0.85      0.89      0.87        44
                 Condition       0.62      0.64      0.63        88
Other Measurement / Result       0.48      0.49      0.48        47
                      Date       0.72  

In [92]:
tr_report = classification_report(y_train,tr_pred)
print("Train Results:\n","-"*60)
print(tr_report)
print("-"*60)

Train Results:
 ------------------------------------------------------------
                            precision    recall  f1-score   support

                 Procedure       0.98      0.98      0.98       402
             O2 Saturation       1.00      0.94      0.97        93
                       GEO       0.96      0.96      0.96       493
                       DOS       0.97      0.97      0.97      1465
                      Time       0.97      0.97      0.97       225
                      Dose       1.00      1.00      1.00       239
          Test / Screening       0.99      0.99      0.99       351
Other Measurement / Result       0.97      0.97      0.97       421
                 Condition       0.98      0.98      0.98       782
        Patient Relocation       0.91      0.94      0.92       309
                       POI       1.00      0.99      0.99       138
                      Drug       1.00      0.99      1.00       679
                       Age       0.98 

Here we can see some of the weights used by the CRF in determining label classes.

In [63]:
eli5.show_weights(crf,top=30)

From \ To,O,B-Accident / Event,I-Accident / Event,B-Age,I-Age,B-BODY,I-BODY,B-Blood Pressure,I-Blood Pressure,B-Condition,I-Condition,B-DOS,I-DOS,B-Date,I-Date,B-Dose,I-Dose,B-Drug,I-Drug,B-Duration,I-Duration,B-Frequency,I-Frequency,B-GEO,I-GEO,B-Gender,B-Heart Rate,I-Heart Rate,B-O2 Saturation,I-O2 Saturation,B-Other Measurement / Result,I-Other Measurement / Result,B-POI,I-POI,B-Patient Relocation,I-Patient Relocation,B-Procedure,I-Procedure,B-Quantity,I-Quantity,B-Respiratory Rate,I-Respiratory Rate,B-Route,I-Route,B-Temperature,I-Temperature,B-Test / Screening,I-Test / Screening,B-Time,I-Time
O,3.831,3.512,0.0,3.242,0.0,3.039,0.0,3.43,0.0,3.491,0.0,3.648,0.0,1.652,0.0,2.543,0.0,3.552,0.0,3.088,0.0,2.042,0.0,3.096,0.0,2.394,2.972,0.0,3.246,0.0,3.101,0.0,3.388,0.0,3.028,0.0,3.117,0.0,2.445,0.0,2.534,0.0,1.21,0.0,3.0,0.0,2.913,0.0,2.208,0.0
B-Accident / Event,-0.425,0.0,6.584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.18,0.0,0.247,0.0,0.0,0.0,0.803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.793,0.0
I-Accident / Event,-0.252,0.0,7.248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038,0.0
B-Age,0.689,0.0,0.0,0.0,4.816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-Age,0.453,0.0,0.0,0.0,4.965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-BODY,-0.048,0.0,0.0,0.0,0.0,0.0,7.198,0.0,0.0,1.755,0.0,2.537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.162,0.0,0.0,0.0
I-BODY,-0.571,0.0,0.0,0.0,0.0,0.0,6.823,0.0,0.0,1.301,0.0,1.879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.476,0.0,-0.006,0.0
B-Blood Pressure,-0.636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-Blood Pressure,0.536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.906,0.0,0.0,0.526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.53,0.0,0.0,0.0,0.0,0.0,0.194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-Condition,0.478,0.0,0.0,0.0,0.0,0.41,0.0,0.0,0.0,0.0,7.813,0.505,0.0,1.271,0.0,0.245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.788,0.0,0.0,0.0,1.09,0.0,0.0,0.0,-0.696,0.0,0.051,0.0,0.181,0.0,0.0,0.0,1.138,0.0,0.0,0.0,0.0,0.0,-0.204,0.0


In [88]:
model_f1 = f1_score(y_test,pred)
print("F1-Score:",model_f1)

F1-Score: 0.6461758398856327


The model yields a respectable F1-Score compared to the other models. Only the BiLSTM with Custom Word2Vec Embeddings is able to outperform it and it does so only marginally.

### 7. Saving Model Results

In [37]:
n_samples = len(x)
model_desc = f"CRF"
results_file = "./data/model_results.csv"
note = '''Simple CRF model with same encodings as Word2Vec Models'''
def append_model_results(model_f1,n_samples,model_desc,file,note):
    with open(file,'a') as f:
        results = f"\n{model_f1},{int(n_samples)},{model_desc},{time.ctime()},{note}"
        f.writelines(results)
    print("~~~Results Successfully Saved")
    return

In [38]:
append_model_results(model_f1,n_samples,model_desc,results_file,note)

~~~Results Successfully Saved


In [39]:
pd.read_csv(results_file).tail()

,F1-Score,n-sample,Model Type,Date,Notes
1,0.587530,1008,BiLSTM-Word2Vec-EmbedSize-100,Tue Nov 12 13:45:40 2019,Custom Word2Vec of entire corpus with numeric ...
2,0.608212,2009,BiLSTM-Word2Vec-EmbedSize-100,Thu Nov 14 13:07:01 2019,Custom Word2Vec of entire corpus with numeric ...
3,0.618051,2009,BiLSTM-Word2Vec-EmbedSize-100,Thu Nov 14 13:14:45 2019,Same as last with bug fix for slicing token co...
4,0.628214,2009,BiLSTM-Word2Vec-EmbedSize-100,Thu Nov 14 13:25:56 2019,20 Epochs no validation
5,0.650725,2009,CRF,Thu Nov 14 13:31:06 2019,Simple CRF model with same encodings as Word2V...


---